In [2]:
import os
import shutil

References: 
1. https://www.geeksforgeeks.org/python-move-all-files-from-subfolders-to-main-folder/
2. https://stackoverflow.com/questions/123198/how-to-copy-files
3. https://pypi.org/project/split-folders/
4. https://www.geeksforgeeks.org/create-a-directory-in-python/
5. https://www.geeksforgeeks.org/python-check-if-a-file-or-directory-exists-2/

In [3]:
# DOMAIN B: COLOR 
root = "ncd"
domain = "ColorfulOriginal"
src = os.path.join(root, domain)
sub_directories = os.listdir(src)

dest = "color_all"
isDir = os.path.isdir(dest) 
if not isDir:
    os.mkdir(dest) 


for d in sub_directories:
    sub_directory = os.path.join(root, domain, d)
    files = os.listdir(sub_directory)
    for file in files:
        file_name = os.path.join(sub_directory, file)
        shutil.copy(file_name, dest)


print(len(os.listdir(dest)))

721


In [4]:
# put them all in the same domain folder

# DOMAIN A: GRAY 
root = "ncd"
domain = "Gray"
src = os.path.join(root, domain)
sub_directories = os.listdir(src)

dest = "gray_all_paired"
isDir = os.path.isdir(dest) 
if not isDir:
    os.mkdir(dest) 


for d in sub_directories:
    sub_directory = os.path.join(root, domain, d)
    files = os.listdir(sub_directory)
    for file in files:
        file_name = os.path.join(sub_directory, file)
        # contains a pair in colored dataset
        is_paired = os.path.isfile(os.path.join("color_all", file))
        if is_paired:
            shutil.copy(file_name, dest)
        else:

            dest_alternate = "gray_all_unpaired"
            isDir = os.path.isdir(dest_alternate) 
            if not isDir:
                os.mkdir(dest_alternate) 
            shutil.copy(file_name, dest_alternate)

print(len(os.listdir(dest)))
print(len(os.listdir(dest_alternate)))

721
15


In [5]:
# create directories
# DOMAIN B: COLOR
dests_color = ["ncdataset/color/train", "ncdataset/color/test", "ncdataset/color/val"]
for d in dests_color:
    isDir = os.path.isdir(d) 
    if not isDir:
        os.makedirs(d) 
# DOMAIN A: GRAY       
dests_gray = ["ncdataset/gray/train", "ncdataset/gray/test", "ncdataset/gray/val"]
for d in dests_gray:
    isDir = os.path.isdir(d) 
    if not isDir:
        os.makedirs(d) 

In [6]:
# NEED PAIRS FOR ALL (TEST, VAL, and TRAIN) SO COMMENTED THIS OUT
# if they don't have a pair, then they are used for testing (15)
# files = os.listdir("gray_all_unpaired")
# for file in files:
#     path = os.path.join("gray_all_unpaired", file)
#     shutil.copy(path, dests_gray[1])

Deleting existing files: https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder

In [7]:
import glob 

# gets rid of all files in specified path
def empty_directories(path_arr):
    for path in path_arr:
        files = glob.glob(os.path.join(path, '*'))
        if len(files) > 0 :
            for f in files:
                os.remove(f)

In [22]:
def loop_and_copy(num_files, color_files, gray_files, dst_path_gray, dst_path_color):
    for i in range(num_files):
    # choose randomly, seed for deterministic behavior
        color_file = random.Random(seed).choice(color_files)
        gray_file = color_file

        # account for discrepancies in capitalization
        if gray_file not in gray_files:
            if gray_file[0].isupper():
                gray_file = gray_file.lower()
                # print('lower', gray_file)
            else:
                gray_file = gray_file.capitalize()
                # print('upper', gray_file)
        # remove from list
        color_files.remove(color_file)
        gray_files.remove(gray_file)

        path_gray = os.path.join("gray_all_paired", gray_file)
        path_color = os.path.join("color_all", color_file)
        # copy files in both directories
        shutil.copy(path_gray, dst_path_gray)
        shutil.copy(path_color, dst_path_color)

Selecting a random file: https://stackoverflow.com/questions/25096158/copying-random-files-from-a-file-tree

In [23]:
# put remaining 80% in train and 20% in val
import random
import math 

train_ratio = 0.8
val_ratio = 0.1

seed = 100

files_color = os.listdir("color_all")
files_gray = os.listdir("gray_all_paired")


path_gray_train = "ncdataset/gray/train"
path_color_train = "ncdataset/color/train"
path_gray_val = "ncdataset/gray/val"
path_color_val = "ncdataset/color/val"
path_gray_test = "ncdataset/gray/test"
path_color_test = "ncdataset/color/test"

# make sure they are truly paired
assert len(files_gray) == len(files_color) 

# empty destination directories if they aren't already   
empty_directories(dests_color)
empty_directories(dests_gray)

# get necessary initial values
num_total = len(files_color)
num_train = math.ceil(train_ratio * num_total)
num_val = math.ceil(val_ratio * num_total)
num_test = num_total - (num_val + num_train)

print('Number of training samples:', num_train)
print('Number of validation samples:', num_val)
print('Number of testing samples:', num_test)

# loop and copy into each respective folder
loop_and_copy(num_train, files_color, files_gray, path_gray_train, path_color_train)
loop_and_copy(num_val, files_color, files_gray, path_gray_val, path_color_val)
loop_and_copy(num_test, files_color, files_gray, path_gray_test, path_color_test)

# len(files_gray)

Number of training samples: 577
Number of validation samples: 73
Number of testing samples: 71


In [24]:
# cleaning up
paths = ['color_all', 'gray_all_paired']
for p in paths:
    path = p
    if os.path.exists(path):
        shutil.rmtree(path)

In [ ]:
# was going to utilize this but not exactly what I want, could integrate it in future to lower the amount of work
# import splitfolders

# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("ncd/Gray", output="out",
#     seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values